In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Face Mesh module
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.3)

# Initialize the video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

def calculate_emotion(landmarks, image_width, image_height):
    # Core facial landmarks for normalization
    left_eye_inner = landmarks[133]
    right_eye_inner = landmarks[362]
    eye_distance = np.linalg.norm(np.array(left_eye_inner) - np.array(right_eye_inner))

    # Eye Openness
    left_eye_top = landmarks[159]  # Check MediaPipe index
    left_eye_bottom = landmarks[145]  # Check MediaPipe index
    right_eye_top = landmarks[386]  # Check MediaPipe index
    right_eye_bottom = landmarks[374]  # Check MediaPipe index
    left_eye_openness = np.linalg.norm(np.array(left_eye_top) - np.array(left_eye_bottom)) / eye_distance
    right_eye_openness = np.linalg.norm(np.array(right_eye_top) - np.array(right_eye_bottom)) / eye_distance
    average_eye_openness = (left_eye_openness + right_eye_openness) / 2

    # Nose Wrinkle
    nose_top = landmarks[6]  # Check MediaPipe index
    nose_bottom = landmarks[4]  # Check MediaPipe index
    nose_wrinkle = np.linalg.norm(np.array(nose_top) - np.array(nose_bottom)) / eye_distance

    # Lip Corner Puller
    lip_left_corner = landmarks[61]  # Check MediaPipe index
    lip_right_corner = landmarks[291]  # Check MediaPipe index
    lip_corner_distance = np.linalg.norm(np.array(lip_left_corner) - np.array(lip_right_corner)) / eye_distance

    # Chin Raiser
    chin_bottom = landmarks[152]  # Check MediaPipe index
    lip_bottom = landmarks[17]  # Check MediaPipe index
    chin_raiser = np.linalg.norm(np.array(chin_bottom) - np.array(lip_bottom)) / eye_distance

    # Lip Pressor
    lip_upper = landmarks[13]  # Check MediaPipe index
    lip_lower = landmarks[14]  # Check MediaPipe index
    lip_pressure = np.linalg.norm(np.array(lip_upper) - np.array(lip_lower)) / eye_distance
    print(f'Eye Openness: {average_eye_openness:.2f}, Nose Wrinkle: {nose_wrinkle:.2f}, Lip Corner Distance: {lip_corner_distance:.2f}, Chin Raiser: {chin_raiser:.2f}, Lip Pressure: {lip_pressure:.2f}')
    # Emotion Detection Logic with additional feature
    if lip_corner_distance > 1.5 and average_eye_openness > 0.2 and lip_pressure <  0.4:
        return "Smiling"
    elif nose_wrinkle < 0.87 and lip_pressure > 0.1 and lip_corner_distance < 1.55  and average_eye_openness< 0.26:
        return "Disgusted"
    elif lip_pressure <= 0.02 and average_eye_openness < 0.23:
        return "Sad"
    elif average_eye_openness > 0.25 and lip_pressure > 0.2 :
        return "Surprised"
    else:
        return "Neutral"

# Continue with your webcam capture and processing logic as before

while cap.isOpened():
    success, image = cap.read()
    original_image = image.copy()
    if not success:
        continue

    # Convert the BGR image to RGB and upscale
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_rgb = cv2.resize(image_rgb, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)

    # Process it with MediaPipe Face Mesh
    results = face_mesh.process(image_rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Draw the face mesh annotations on the image.
            mp.solutions.drawing_utils.draw_landmarks(
                image=image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_CONTOURS,
                landmark_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1),
                connection_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(color=(0, 255, 0), thickness=1))

            # Convert landmarks to an array for easier manipulation
            landmark_array = np.array([(lm.x * image.shape[1], lm.y * image.shape[0]) for lm in face_landmarks.landmark])

            # Detect emotion
            emotion = calculate_emotion(landmark_array, image.shape[1], image.shape[0])

            # Display detected emotion
            cv2.putText(image, f'Emotion: {emotion}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Display the image
    # Concatenate original and processed image horizontally
    comparison_frame = np.hstack((original_image, image))

    # Display the image
    cv2.imshow('Original vs Processed Emotion Detection', comparison_frame)
    if cv2.waitKey(5) & 0xFF == 27:
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()
